In [1]:
#$ Entity Masking
import json
from span_marker import SpanMarkerModel


def load_entities_mapping(file_path):
    """
    Loads entities and their translations from the given JSON file, skipping invalid lines.
    Args:
        file_path (str): Path to the JSON file containing entity mappings.
    Returns:
        list: A list of dictionaries representing the JSON data.
    """
    with open(file_path, 'r') as jf:
        try:
            return json.load(jf)  # Load the entire JSON array
        except json.JSONDecodeError as e:
            print(f"Error loading JSON file: {e}")
            return []


def mask_entities_with_spanmarker(text, model):
    """
    Masks named entities in text using predictions from SpanMarkerModel.

    Args:
        text (Union[str, List[str]]): Input text(s). Can be a single string or a list of strings.
        model (SpanMarkerModel): Pretrained SpanMarker model for entity prediction.

    Returns:
        Union[Tuple[str, dict], List[Tuple[str, dict]]]:
            - For a single string:
                - masked_text (str): Text with entities replaced by placeholders.
                - mapping (dict): Mapping of placeholders to original entities.
            - For a list of strings:
                - List of (masked_text, mapping) tuples.
    """
    # Check if input is a single string or a list of strings
    is_single_string = isinstance(text, str)
    if is_single_string:
        text = [text]  # Convert single string to list for uniform processing

    # Perform predictions using the model
    predictions = model.predict(text)  # Handles batch processing

    results = []
    for i, sentence in enumerate(text):
        masked_text = sentence
        mapping = {}
        type_counters = {}
        offset = 0

        # If no predictions for this sentence
        if not predictions[i]:  # Empty list
            print(f"No entities found for text: {sentence}")
            results.append((masked_text, mapping))
            continue

        # Process each entity in the prediction
        for entity in predictions[i]:
            entity_text = entity['span']
            entity_type = entity['label'].upper()
            start = entity['char_start_index']
            end = entity['char_end_index']

            # Counter for entity type
            if entity_type not in type_counters:
                type_counters[entity_type] = 1
            else:
                type_counters[entity_type] += 1

            # Generate a placeholder
            placeholder = f"[ENTITY_{entity_type}_{type_counters[entity_type]}]"

            # Replace entity with placeholder in the text
            start += offset
            end += offset
            masked_text = masked_text[:start] + placeholder + masked_text[end:]
            offset += len(placeholder) - len(entity_text)

            # Add to mapping
            mapping[placeholder] = {
                "original_text": entity_text,
                "type": entity_type,
                "confidence": entity['score'],
                "start": start,
                "end": end
            }

        results.append((masked_text, mapping))

    #if text input is a single string
    return results[0] if is_single_string else results

def ensure_pad_token(model):
    if not hasattr(model.tokenizer, "pad_token") or model.tokenizer.pad_token is None:
        print("Setting pad_token manually...")
        model.tokenizer.pad_token = "[PAD]"
        model.tokenizer.pad_token_id = model.tokenizer.convert_tokens_to_ids("[PAD]")

In [2]:
#$ MT Translation

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the M2M100 model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

def translate_with_placeholders_m2m(masked_text, mapping, src_lang, tgt_lang):
    """
    Translates the non-entity parts of a text and re-inserts placeholders using M2M100.

    Args:
        masked_text (str): Text with placeholders for entities.
        mapping (dict): Mapping of placeholders to entity details.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: Translated text with placeholders intact.
    """
    tokenizer.src_lang = src_lang

    # Encode and translate the entire sentence
    encoded_text = tokenizer(masked_text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang),
        max_length=128,  # Ensure sufficient length for translation
        early_stopping=True
    )

    # Decode the translated sentence
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text




In [3]:
#$ Entity Reintegration

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

def translate_entity(entity, src_lang, tgt_lang):
    """
    Translates an entity if its type requires translation, otherwise returns the original entity.

    Args:
        entity (str): The placeholder or entity to process.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: The translated or original entity.
    """
    # types that should not be translated
    non_translatable_types = {"PER", "INST", "MEDIA", "PLANT", "VEHI"}

    # Check if the entity contains one of the non-translatable types
    if any(ntype in entity for ntype in non_translatable_types):
        return entity 

    # Translate the entity using M2M100
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
    model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    tokenizer.src_lang = src_lang
    encoded_entity = tokenizer(entity, return_tensors="pt")
    generated_tokens = model.generate(**encoded_entity, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    translated_entity = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_entity


def reintegrate_entities(translation, mapping, src_lang, tgt_lang):
    """
    Replaces placeholders in the translated text with original or translated entities.

    Args:
        translation (str): Translated text with placeholders.
        mapping (dict): Mapping of placeholders to original entities.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: Translation with entities reintegrated.
    """
    for placeholder, entity_info in mapping.items():
        # Get the original entity
        original_entity = entity_info["original_text"]

        # Translate the entity if needed
        translated_entity = translate_entity(original_entity, src_lang, tgt_lang)

        # Replace placeholder with the translated entity
        translation = translation.replace(placeholder, translated_entity)

    return translation


In [4]:
import os
it_datapath = os.path.abspath('data/italian_test.json')

    # Step 1: Load the data
print("Loading Italian data...")
italian_data = load_entities_mapping(it_datapath)

Loading Italian data...


In [5]:
italian_data

[{'id': '86fb5e94',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'How old is the author of the Goosebumps series?',
  'target': "Quanti anni ha l'autore della serie Piccoli Brividi?",
  'entities': {'Q3260367': {'it': 'Piccoli brividi', 'en': 'Goosebumps'}},
  'from': 'mintaka'},
 {'id': '14b9185d',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'Which of the three largest islands in the Mediterranean Sea is not part of Italy?',
  'target': 'Quale delle tre isole più grandi del Mediterraneo non è italiana?',
  'entities': {'Q38': {'it': 'Italia', 'en': 'Italy'}},
  'from': 'mintaka'},
 {'id': 'ff37d660',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'What house did Severus Snape belong to?',
  'target': 'Qual è la casa di Severus Piton?',
  'entities': {'Q176772': {'it': 'Severus Piton', 'en': 'Severus Snape'}},
  'from': 'mintaka'},
 {'id': 'ff9a84d2',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'Who is the current pres

In [6]:
print("Loading SpanMarker model...")
spanmarker_model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")
ensure_pad_token(spanmarker_model)

Loading SpanMarker model...


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Setting pad_token manually...


In [7]:
print("Loading M2M100 translation model...")
m2m_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
m2m_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

Loading M2M100 translation model...


In [8]:
translated_sentences = []

for idx, entry in enumerate(italian_data):  # Process only the first 10 entries

    # Get source text (English) and target text (Italian)
    eng_text = entry['source']
    itl_text = entry['target']

    # Mask entities in the English text
    masked_eng_text, mapping = mask_entities_with_spanmarker(eng_text, spanmarker_model)
    print(f"Masked English Text: {masked_eng_text}")
    print(f"Entity Mapping: {json.dumps(mapping, indent=4)}")

    # Filter entities by confidence score
    filtered_mapping = {
        key: value for key, value in mapping.items() if value["confidence"] > 0.8
    }
    print(f"Filtered Mapping (Confidence > 0.8): {json.dumps(filtered_mapping, indent=4)}")

    # Translate non-entity parts of the text
    m2m_tokenizer.src_lang = "en"
    tgt_lang = "it"
    translated_text_with_placeholders = translate_with_placeholders_m2m(
        masked_eng_text, filtered_mapping, src_lang="en", tgt_lang="it"
    )
    print(f"Translated Text with Placeholders: {translated_text_with_placeholders}")

    # Reintegrate entities into the translated text
    final_translation = reintegrate_entities(
        translated_text_with_placeholders, filtered_mapping, src_lang="en", tgt_lang="it"
    )
    print(f"Final Translation: {final_translation}")

    # Append final translation to the results
    translated_sentences.append({
        "id": entry["id"],
        "source": eng_text,
        "target": itl_text,
        "masked_source": masked_eng_text,
        "translation": final_translation,
        "filtered_mapping": filtered_mapping
    })

# Save the results
results_path = "data/translated_italian_test.json"
with open(results_path, "w") as results_file:
    json.dump(translated_sentences, results_file, indent=4)

print(f"Translation results for test set saved to {results_path}")


Masked English Text: How old is the author of the [ENTITY_MEDIA_1] series?
Entity Mapping: {
    "[ENTITY_MEDIA_1]": {
        "original_text": "Goosebumps",
        "type": "MEDIA",
        "confidence": 0.9521344304084778,
        "start": 29,
        "end": 39
    }
}
Filtered Mapping (Confidence > 0.8): {
    "[ENTITY_MEDIA_1]": {
        "original_text": "Goosebumps",
        "type": "MEDIA",
        "confidence": 0.9521344304084778,
        "start": 29,
        "end": 39
    }
}
Translated Text with Placeholders: Quanto è vecchio l'autore della serie [ENTITY_MEDIA_1]?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quanto è vecchio l'autore della serie Goosebumpi?
Masked English Text: Which of the three largest islands in the [ENTITY_LOC_1] is not part of [ENTITY_LOC_2]?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "Mediterranean Sea",
        "type": "LOC",
        "confidence": 0.9999574422836304,
        "start": 42,
        "end": 59
    },
    "[ENTITY_LOC_2]": {
        "original_text": "Italy",
        "type": "LOC",
        "confidence": 0.9999597072601318,
        "start": 72,
        "end": 77
    }
}
Filtered Mapping (Confidence > 0.8): {
    "[ENTITY_LOC_1]": {
        "original_text": "Mediterranean Sea",
        "type": "LOC",
        "confidence": 0.9999574422836304,
        "start": 42,
        "end": 59
    },
    "[ENTITY_LOC_2]": {
        "original_text": "Italy",
        "type": "LOC",
        "confidence": 0.9999597072601318,
        "start": 72,
        "end": 77
    }
}
Translated Text with Placeholders: Quale delle tre isole più gr

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quale delle tre isole più grandi della Mar Mediterraneo non fa parte della Italia?
Masked English Text: What house did [ENTITY_PER_1] belong to?
Entity Mapping: {
    "[ENTITY_PER_1]": {
        "original_text": "Severus Snape",
        "type": "PER",
        "confidence": 0.9997941851615906,
        "start": 15,
        "end": 28
    }
}
Filtered Mapping (Confidence > 0.8): {
    "[ENTITY_PER_1]": {
        "original_text": "Severus Snape",
        "type": "PER",
        "confidence": 0.9997941851615906,
        "start": 15,
        "end": 28
    }
}
Translated Text with Placeholders: A quale casa appartiene [ENTITY_PER_1]?


...

Final Translation: È Parigi il capitale di Francia?
Masked English Text: Which Lord of the Rings character is a relative to [ENTITY_MYTH_1] and originally finds the ring?
Entity Mapping: {
    "[ENTITY_MYTH_1]": {
        "original_text": "Frodo",
        "type": "MYTH",
        "confidence": 0.8314065933227539,
        "start": 51,
        "end": 56
    }
}
Filtered Mapping (Confidence > 0.8): {
    "[ENTITY_MYTH_1]": {
        "original_text": "Frodo",
        "type": "MYTH",
        "confidence": 0.8314065933227539,
        "start": 51,
        "end": 56
    }
}
Translated Text with Placeholders: Quale personaggio del Signore degli Anelli è relativo a [ENTITY_MYTH_1] e trova originariamente l'anello?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quale personaggio del Signore degli Anelli è relativo a di Frodo e trova originariamente l'anello?
Masked English Text: Who ruled [ENTITY_LOC_1] after [ENTITY_PER_1], but did not live to 40?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "Egypt",
        "type": "LOC",
        "confidence": 0.9998869895935059,
        "start": 10,
        "end": 15
    },
    "[ENTITY_PER_1]": {
        "original_text": "Cleopatra",
        "type": "PER",
        "confidence": 0.9998526573181152,
        "start": 31,
        "end": 40
    }
}
Filtered Mapping (Confidence > 0.8): {
    "[ENTITY_LOC_1]": {
        "original_text": "Egypt",
        "type": "LOC",
        "confidence": 0.9998869895935059,
        "start": 10,
        "end": 15
    },
    "[ENTITY_PER_1]": {
        "original_text": "Cleopatra",
        "type": "PER",
        "confidence": 0.9998526573181152,
        "start": 31,
        "end": 40
    }
}
Translated Text with Placeholders: Chi ha gover

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Chi ha governato Egitto dopo di Cleopatra, ma non ha vissuto fino a 40?
Translation results for test set saved to data/translated_italian_test.json
